In [ ]:
import os
exists = os.path.isfile('readtria.py')
if not(exists):
    print("Downloading meshes ...")
    !git clone https://github.com/dpeschka/NumFluids.git
    !cp -r NumFluids/notebook/meshes .
    !cp NumFluids/notebook/readtria.py .
else:
    print("All is set.")

# General functions for all examples

These functions extend the code of the previous lecture in the following sense:

* more flexible tensor mesh
* local stiffness matrix with space-dependent coefficient $a(x)$
* local mass matrix with space-dependent coefficient $c(x)$
* convection matrix with space-dependent vector $\vec{u}(x)$
* output into vtk files for visualization in paraview (see `readtria.py`)

The corresponding functions are streamlined a little bit, i.e. `np.matmul` is used consistently.

In [ ]:
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import spsolve,splu
from scipy.spatial import Delaunay

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from readtria import readtria, writevtk

def square_mesh(L=1.0,k=16):
    h = L/k
    xh, yh = np.meshgrid(np.linspace(0,L,k), np.linspace(0,L,k)) 
    xh     = xh.flatten()
    yh     = yh.flatten()
    points = np.array([xh,yh]).T
    tri    = Delaunay(points)
    e2     = tri.simplices
    
    nelement = np.size(e2,0)
    npoint   = np.size(xh,0)
    
    idp      = np.zeros(npoint)
    idp[(xh<h/2)|(yh<h/2)|(xh>(L-h/2))|(yh>(L-h/2))] = 1
    ide      = []
    return xh,yh,npoint,nelement,e2,idp,ide

def tensor_mesh(Lx,Ly):
    eeps = 1e-8
    x0 = Lx[ 0]+eeps
    x1 = Lx[-1]-eeps
    y0 = Ly[ 0]+eeps
    y1 = Ly[-1]-eeps
    xh, yh = np.meshgrid(Lx,Ly) 
    xh     = xh.flatten()
    yh     = yh.flatten()
    points = np.array([xh,yh]).T
    tri    = Delaunay(points)
    e2     = tri.simplices
    
    nelement = np.size(e2,0)
    npoint   = np.size(xh,0)
    
    idp      = np.zeros(npoint)
    idp[(xh<x0)|(yh<y0)|(xh>x1)|(yh>y1)] = 1
    ide      = []
    return xh,yh,npoint,nelement,e2,idp,ide

def generate_transformation2D(k, e2, x, y):
    dx1 = x[e2[k, 1]]-x[e2[k, 0]]
    dy1 = y[e2[k, 1]]-y[e2[k, 0]]

    dx2 = x[e2[k, 2]]-x[e2[k, 0]]
    dy2 = y[e2[k, 2]]-y[e2[k, 0]]

    # determinant on each triangle
    Fdet = dx1*dy2 - dx2*dy1

    # transformation Jacobian on each triangle
    Finv = np.zeros((2, 2))
    Finv[0, 0] =  dy2 / Fdet
    Finv[0, 1] = -dx2 / Fdet
    Finv[1, 0] = -dy1 / Fdet
    Finv[1, 1] =  dx1 / Fdet

    return Fdet, Finv

# computes the local stiffness matrix in 2D, i.e. int grad(w_i)*grad(w_j) dx
# by transformation to reference, where the shape functions phi are used
def local_stiff2D(a,Fdet, Finv):
    gradphi = np.array([[-1, -1], [1, 0], [0, 1]]).T
    dphi    = np.matmul(Finv.T,gradphi)
    S = a/2*np.matmul(dphi.T,dphi)*Fdet
    return S

# computes the local convection matrix in 2D, i.e. int w_i u*grad(w_j) dx
def local_conv2D(uvec,Fdet, Finv):
    locmass = np.array([[1, 1/2, 1/2], [1/2, 1, 1/2], [1/2, 1/2, 1]])/12.0
    gradphi = np.array([[-1, -1], [1, 0], [0, 1]]).T
    dphi    = np.matmul(Finv.T,gradphi)
    C = np.matmul(np.matmul(locmass,uvec),dphi)*Fdet
    return C

# computes the local mass matrix in 2D, i.e. int w_i w_j dx
def local_mass2D(c,Fdet):
    return c*Fdet*np.array([[1, 1/2, 1/2], [1/2, 1, 1/2], [1/2, 1/2, 1]])/12

# Lecture 05, Part 1

The example below shows a stationary solution of a convection diffusion problem

$$
-a\nabla^2 w + \vec{u}\cdot\nabla w + cw = f
$$

on $\Omega=[0,1]^2$. The (lambda) function $a,\vec{u},c,f$ depend on $(x,y)$ and can be specified by you. Note that the solution potentially (depending on the choice of functions) can have a boundary layer. Usually this happens if $0<a\ll 1$ and $\nu\cdot\vec{u}>0$ on the boundary with (outer) normal $\nu$. Such a boundary layer can be resolved using a higher mesh resolution near the boundary, e.g. via the `tensor_mesh` mesh generation for rectangular meshes.

**Tasks and questions:**

1. Experiment with different meshes, different boundary conditions, different RHS. Also, try to change the coefficients $a(x)$, $c(x)$ and $\vec{u}(x)$.

2. What boundary conditions hold at a Neumann boundary depending on the choice of 

$$
\int_\Omega \phi_i \vec{u}\cdot\nabla\phi_j\mathrm{d}x \quad\text{vs}\quad \int_\Omega (-1)\phi_j \vec{u}\cdot\nabla\phi_i\mathrm{d}x
$$

when assuming $\nabla\cdot\vec{u}=0$?

Hint: Via `readtria` you can read the following meshes from `./meshes`

* `disc1`,`disc2`,`disc3`,`disc4` which are increasingly fine version of $\{(x,y)\in\mathbb{R^2}:x^2+y^2\le 1\}$.
* `haus` a supercoarse test mesh constructed by hand
* `house.1` a nice mesh resembling a 2D house
* `box` a unit disc (better use `tensor_mesh` or `square_mesh`)
* `mesh1` a coarse annulus
* `mesh2` a coarse disc
* `mesh3` a coarse square
* `cheese` a domain with smooth boundary and some holes
* `domain_disc` currently incompatible

In [ ]:
import time

# FEM Python sample code
#x0 = 0.975
#Ly = np.linspace(0,1,96)
#Lx = np.append(np.linspace(0,x0,64)[:-1],np.linspace(x0,1,32))
#x,y,npoint,nelement,e2,idp,ide = tensor_mesh(Lx,Ly)

x,y,npoint,nelement,e2,idp,ide = readtria('./meshes/disc3')
dofmap = e2

# select points without Dirichlet bc
it = np.logical_not(idp == 1)
nphi = 3

# build matrices
ii = np.zeros((nelement, nphi**2))  # sparse i-index
jj = np.zeros((nelement, nphi**2))  # sparse j-index
aa = np.zeros((nelement, nphi**2))  # entry of Galerkin matrix
bb = np.zeros((nelement, nphi**2))  # entry in mass-matrix (to build rhs)

tstart = time.perf_counter()

for k in range(nelement):
    Fdet, Finv = generate_transformation2D(k, e2, x, y)  # compute trafo
    
    # build local matrices (mass, stiffness, ...)
    a = lambda x,y:  0.1 + 0*x.T
    c = lambda x,y:  0.0 + 0*x.T
    u = lambda x,y: np.array((1+0*x,0*x)).T
    
    # center points (xm,ym) and nodal points (xe,ye) of element
    xe   = x[e2[k,:]]
    ye   = y[e2[k,:]]
    xm   = np.mean(xe)
    ym   = np.mean(ye)
    
    sloc  = local_stiff2D(a(xm,ym),Fdet,Finv) # element stiffness matrix
    mloc  = local_mass2D (c(xm,ym),Fdet     ) # element mass matrix
    cloc  = local_conv2D (u(xe,ye),Fdet,Finv) # element convection matrix
    mlocR = local_mass2D (1.0     ,Fdet     ) # element mass matrix
    
    # compute i,j indices of the global matrix
    dofs     = dofmap[k,:]
    ii[k, :] = dofs[[0,0,0,1,1,1,2,2,2]]  # local-to-global
    jj[k, :] = dofs[[0,1,2,0,1,2,0,1,2]]  # local-to-global
    
    # compute a(i,j) values of the global matrix
    aloc = sloc + cloc + mloc
    aa[k, :] = aloc.flatten()
    bb[k, :] = mlocR.flatten()
    
tend = time.perf_counter()
print(f"Elapsed (assembly) = {tend-tstart:9.4f} sec.")

# create sparse matrices
tstart = time.perf_counter()
L = sparse.csc_matrix((aa.flatten(),(ii.flatten(),jj.flatten())),shape=(npoint,npoint))
M = sparse.csc_matrix((bb.flatten(),(ii.flatten(),jj.flatten())),shape=(npoint,npoint))
tend   = time.perf_counter()
print(f"Elapsed (matrix)   = {tend-tstart:9.4f} sec.")

# build rhs and take into account Dirichlet bcs
f     = lambda x,y:  1+0*x
rhs   = M*f(x,y)
w     = 0*x
L1    = L[it,:]
L2    = L1[:,it]

# solve
tstart = time.perf_counter()
w[it]  = spsolve(L2,rhs[it])
tend   = time.perf_counter()
print(f"Elapsed (solver)   = {tend-tstart:9.4f} sec.")

# plot
plt.figure(figsize=(8,8))
plt.tripcolor(x,y,w,triangles=e2,shading='gouraud')
plt.triplot(x,y,triangles=e2,linewidth=0.05)
plt.axis('equal')

# Neumann Problem

We want to solve the problem

$$
-\nabla\cdot(a\nabla w)=f \qquad \text{in }\Omega
$$

with boundary conditions $a\nu\cdot\nabla w=g$ (or even $g=0$) on the entire boundary $\partial\Omega$. This problem turns out to be ill-posed (does not have a solution in general, and if so, the solution is not unique). This can be seen as follows:

### 1. Failure of existence

By integrating the PDE over the domain you get
$$
\int_\Omega f\,\mathrm{d}x=\int_\Omega -\nabla\cdot(a\nabla w)\,\mathrm{d}x=\int_{\partial\Omega} -a\nu\cdot\nabla w \mathrm{d}s = \int_{\partial\Omega} -g\mathrm{d}s
$$
where in the middle part we used Gauss theorem. This implies that a solution can only exist if the data, i.e. the given functions $f$ and $g$ satisfy
$$
\int_\Omega f\,\mathrm{d}x+\int_{\partial\Omega} g\mathrm{d}s=0
$$
and otherwise a solution cannot not exist.

### 2. Failure of uniqueness

For any solution $w$, you can simply obtain a new solution 
$\tilde{w}=w+C$ by adding an arbitrary constant $C\in\mathbb{R}$. This means that the operator $L$ used to write $Lw=f$ has a kernel (zero eigenmode), i.e. a nontrivial function such that $Lw_0=0$. Hence, the operator is not invertible and the inverse matrix $L_h^{-1}$ does not exist.

## Solution: Lagrange multipliers

The solution to that problem is to enforce the constraint $\int_\Omega w\mathrm{d}x=0$ while solving the problem. In the weak form, this leads to a saddle point problem where we seek $u\in V$ and $\lambda\in\mathbb{R}$ that satisfy

$$
\begin{split}
A(u,v) + b(\lambda,v)=\int fv\mathrm{d}x\\
b(\rho,u)=0
\end{split}
$$

where $A(u,v)=\int_\Omega a\nabla u\cdot\nabla v\,\mathrm{d}x$ and $b(\lambda,v)=\lambda\int_\Omega v\mathrm{d}x$. Correspondingly, we can solve the algebraic problem

$$
\bar{L}_{\rm ext}\begin{pmatrix} w \\ \lambda\end{pmatrix}= \begin{pmatrix}
\bar{L} & B^T\\
B & 0\end{pmatrix}\begin{pmatrix} w \\ \lambda\end{pmatrix} = \begin{pmatrix} Mf \\ 0\end{pmatrix}=\mathrm{RHS}_\mathrm{ext}
$$

with Galerkin matrix $\bar{A}_{ij}=A(\varphi_j,\varphi_i)$ and $B_i=b(1,\varphi_i)=\sum_j M_{ij}=(M(1,1,...,1)^T)_i$. Please confirm this identity and check the calculation below.

In [ ]:
# FEM Python sample code
x,y,npoint,nelement,e2,idp,ide = readtria('./meshes/disc2')
dofmap = e2

# select points without Dirichlet bc
it = np.logical_not(idp == 1)
nphi = 3

# build matrices
ii = np.zeros((nelement, nphi**2))  # sparse i-index
jj = np.zeros((nelement, nphi**2))  # sparse j-index
aa = np.zeros((nelement, nphi**2))  # entry of Galerkin matrix
bb = np.zeros((nelement, nphi**2))  # entry in mass-matrix (to build rhs)

for k in range(nelement):
    Fdet, Finv = generate_transformation2D(k, e2, x, y)  # compute trafo
    
    # build local matrices (mass, stiffness, ...)
    a = lambda x,y:  1 + 0*x.T
    
    # center points (xm,ym) and nodal points (xe,ye) of element
    xm   = np.mean(xe)
    ym   = np.mean(ye)
    
    sloc = local_stiff2D(a(xm,ym),Fdet,Finv) # element stiffness matrix
    mlocR= local_mass2D (1.0     ,Fdet     ) # element mass matrix
    
    # compute i,j indices of the global matrix
    dofs     = dofmap[k,:]
    ii[k, :] = dofs[[0,0,0,1,1,1,2,2,2]]  # local-to-global
    jj[k, :] = dofs[[0,1,2,0,1,2,0,1,2]]  # local-to-global
    
    # compute a(i,j) values of the global matrix
    aloc = sloc 
    aa[k, :] = aloc.flatten()
    bb[k, :] = mlocR.flatten()

# create sparse matrices
L = sparse.csc_matrix((aa.flatten(),(ii.flatten(),jj.flatten())),shape=(npoint,npoint))
M = sparse.csc_matrix((bb.flatten(),(ii.flatten(),jj.flatten())),shape=(npoint,npoint))

B    = np.reshape(M*(1+0*x),[np.size(x),1])
BT   = np.reshape(M*(1+0*x),[1,np.size(x)])
Lext = sparse.bmat([[L,B],[BT,0]],format='csc')

# build rhs and take into account Dirichlet bcs, solve, plot
f      = lambda x,y:  x+y*y
rhs    = M*f(x,y)
w      = 0*x

rhs_ext = np.append(rhs,[0])

w  = spsolve(Lext,rhs_ext) # solve problem with Lagrange multiplier
w1 = spsolve(L,rhs)        # solve problem without Lagrange multiplier

plt.figure(figsize=(8,8))
c = plt.tripcolor(x,y,w1,triangles=e2,shading='gouraud')
plt.title('without multiplier')
plt.colorbar(c)
plt.axis('equal')

plt.figure(figsize=(8,8))
c = plt.tripcolor(x,y,w[:-1],triangles=e2,shading='gouraud')
plt.title('with multiplier')
plt.colorbar(c)
plt.axis('equal')

# Lecture 5, Part 2

Let

$$
L w=-a\nabla^2 w + \vec{u}\cdot\nabla w, \qquad a=10^{-4},\quad \vec{u}(\vec{x})=(-y,x)
$$

on $\Omega=B_1(0)=\{(x,y)\in\mathbb{R}^2:x^2+y^2<1\}$. With this we solve the problem

$$
\partial_t w + Lw = 0,
$$

with natural boundary conditions and $w(t=0,\vec{x})=w_0(\vec{x})$ and $w_0(\vec{x})=\tanh(20x)$.

**Tasks and questions:**

1. Experiment with different meshes, different boundary conditions, different RHS. Also, try to change the coefficients $a(x)$, $c(x)$ and $\vec{u}(x)$.

2. Here the output is stored in files `simu_lect06_00000-99999.vtk`. Download the files and try to visualize the solution via Paraview.

In [ ]:
# FEM Python sample code
x,y,npoint,nelement,e2,idp,ide = readtria('./meshes/disc3')  # read mesh from file
dofmap = e2

# select points without Dirichlet bc
it = np.logical_not(idp == 1)
nphi = 3

# build matrices
ii = np.zeros((nelement, nphi**2))  # sparse i-index
jj = np.zeros((nelement, nphi**2))  # sparse j-index
aa = np.zeros((nelement, nphi**2))  # entry of Galerkin matrix
bb = np.zeros((nelement, nphi**2))  # entry in mass-matrix (to build rhs)

for k in range(nelement):
    Fdet, Finv = generate_transformation2D(k, e2, x, y)  # compute trafo
    
    # build local matrices (mass, stiffness, ...)
    a = lambda x,y:  1e-4 + 0*x.T
    c = lambda x,y:  1.0  + 0*x.T
    u = lambda x,y: np.array((-y,x)).T
    
    # center points (xm,ym) and nodal points (xe,ye) of element
    xe   = x[e2[k,:]]
    ye   = y[e2[k,:]]
    xm   = np.mean(xe)
    ym   = np.mean(ye)
    
    sloc = local_stiff2D(a(xm,ym),Fdet,Finv) # element stiffness matrix
    mloc = local_mass2D (c(xm,ym),Fdet     ) # element mass matrix
    cloc = local_conv2D (u(xe,ye),Fdet,Finv) # element convection matrix
    
    # compute i,j indices of the global matrix
    dofs     = dofmap[k,:]
    ii[k, :] = dofs[[0,0,0,1,1,1,2,2,2]]  # local-to-global
    jj[k, :] = dofs[[0,1,2,0,1,2,0,1,2]]  # local-to-global
    
    # compute a(i,j) values of the global matrix
    aloc = sloc + cloc
    aa[k, :] = aloc.flatten()
    bb[k, :] = mloc.flatten()

# create sparse matrices
L = sparse.csc_matrix((aa.flatten(),(ii.flatten(),jj.flatten())),shape=(npoint,npoint))
M = sparse.csc_matrix((bb.flatten(),(ii.flatten(),jj.flatten())),shape=(npoint,npoint))

# build rhs and take into account Dirichlet bcs, solve, plot
w   = np.tanh(20*x)
Nt  = 1000
tau = 2*np.pi/(Nt)

A = M + tau*L
B = M 

ALU = splu(A)

fnum = 0
for k in range(Nt):
    rhs = B*w
    w   = ALU.solve(rhs)
    if (k % 200 == 0):
        writevtk(f'simu_lect06_{fnum:05d}.vtk',x,y,e2,w,'w')
        fnum = fnum + 1

plt.figure()
plt.tripcolor(x,y,w,triangles=e2,shading='gouraud')
plt.triplot(x,y,triangles=e2,linewidth=0.05)
plt.axis('equal')


# Lecture 05, Part 3

In this part we want to lay the foundation to apply all the techniques introduced before to fluid flows. This means that we need to be able to

- Solve constraints.
- Solve (vectorial) systems of equations.
- Understand incompressibility.

Therefore think about the following tasks:

1. Reformulate the code above to solve

$$
-\nabla \cdot (a \nabla \vec{u}) + c \vec{u} = \vec{f}
$$

for given functions $a,c:\Omega\to\mathbb{R}$ and $\vec{f}:\Omega\to\mathbb{R}^d$.

2. Check out the Chorin projection, as it is introduced in the video lectures, i.e. an arbitrary vector field $\vec{u}$ is decomposed

$$
\vec{u} = \vec{u}_{ir} + \vec{u}_0
$$

where the irrotational part obeys $\vec{u}_{ir}=\nabla\phi$ and the remainder is divergence-free $\nabla\cdot\vec{u}_0=0$. This implies

$$
\nabla\cdot \vec{u} = \nabla\cdot(\nabla\phi)
$$

which we can solve using the weak formulation introduced above. This requires using the stiffness matrix (for the Laplace operator) and the convection matrix (to compute the divergence of $\vec{u}$) in the problem formulation. First, one computes $\phi$ via

$$
\int_\Omega \nabla\phi\cdot\nabla v\,\mathrm{d}x = \int_\Omega (-\nabla\cdot\vec{u})v\,\mathrm{d}x.
$$


The mass matrix is required in order to compute the gradient of $\phi$, i.e. we compute $\vec{w}=\nabla\phi$ via

$$
\int \vec{w}\cdot \vec{v}\,\mathrm{d}x=\int_\Omega \vec{v}\cdot\nabla\phi\,\mathrm{d}x,
$$

where on the left side we require a vectorial mass matrix and on the right side the convection matrix.

Below an example, where the divergence of a given vector field is computed.

In [ ]:
# FEM Python sample code
x,y,npoint,nelement,e2,idp,ide = readtria('./meshes/disc2')  # read mesh from file
dofmap = e2

# select points without Dirichlet bc
it = np.logical_not(idp == 1)
nphi = 3

# build matrices
ii = np.zeros((nelement, nphi**2))  # sparse i-index
jj = np.zeros((nelement, nphi**2))  # sparse j-index
aa = np.zeros((nelement, nphi**2))  # entry of Galerkin matrix
bb = np.zeros((nelement, nphi**2))  # entry in mass-matrix (to build rhs)
cx = np.zeros((nelement, nphi**2))  # entry in mass-matrix (to build rhs)
cy = np.zeros((nelement, nphi**2))  # entry in mass-matrix (to build rhs)

for k in range(nelement):
    Fdet, Finv = generate_transformation2D(k, e2, x, y)  # compute trafo
    
    # build local matrices (mass, stiffness, ...)
    a  = lambda x,y:  1.0  + 0*x.T
    c  = lambda x,y:  1.0  + 0*x.T
    ex = lambda x,y: np.array((1+0*x,0*x)).T
    ey = lambda x,y: np.array((0*x,1+0*x)).T
    
    # center points (xm,ym) and nodal points (xe,ye) of element
    xe   = x[e2[k,:]]
    ye   = y[e2[k,:]]
    xm   = np.mean(xe)
    ym   = np.mean(ye)
    
    sloc  = local_stiff2D(a(xm,ym),Fdet,Finv ) # element stiffness matrix
    mloc  = local_mass2D (c(xm,ym),Fdet      ) # element mass matrix
    clocx = local_conv2D (ex(xe,ye),Fdet,Finv) # element convection matrix
    clocy = local_conv2D (ey(xe,ye),Fdet,Finv) # element convection matrix
    
    # compute i,j indices of the global matrix
    dofs     = dofmap[k,:]
    ii[k, :] = dofs[[0,0,0,1,1,1,2,2,2]]  # local-to-global
    jj[k, :] = dofs[[0,1,2,0,1,2,0,1,2]]  # local-to-global
    
    # compute a(i,j) values of the global matrix
    aloc = sloc + cloc
    aa[k, :] = aloc.flatten()
    bb[k, :] = mloc.flatten()
    cx[k, :] = clocx.flatten()
    cy[k, :] = clocy.flatten()

# create sparse matrices
L  = sparse.csc_matrix((aa.flatten(),(ii.flatten(),jj.flatten())),shape=(npoint,npoint))
M  = sparse.csc_matrix((bb.flatten(),(ii.flatten(),jj.flatten())),shape=(npoint,npoint))
Dx = sparse.csc_matrix((cx.flatten(),(ii.flatten(),jj.flatten())),shape=(npoint,npoint))
Dy = sparse.csc_matrix((cy.flatten(),(ii.flatten(),jj.flatten())),shape=(npoint,npoint))

# vector field w=(wx,wy)
wx  = x * np.sqrt(x*x+y*y)
wy  = y * np.sqrt(x*x+y*y)

# divergence of w via weak form
div_w = spsolve(M,Dx*wx + Dy*wy)

# plot divergence and vector field
fig, ax = plt.subplots(figsize=(10,10))
c = plt.tripcolor(x,y,div_w,triangles=e2,shading='gouraud')
plt.triplot(x,y,triangles=e2,linewidth=0.5)
plt.quiver(x,y,wx,wy,scale=30)
plt.axis('equal')
fig.colorbar(c)